In [ ]:
#!pip install mysql-connector-python

In [18]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import mysql.connector
from PIL import Image, ImageTk

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Gestion de Vente")
        self.geometry("800x600")
        self.initialize_database()
        self.create_widgets()

    def initialize_database(self):
        self.db = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="sales"
        )
        self.cursor = self.db.cursor()

        # Créer la table products si elle n'existe pas
        self.cursor.execute("CREATE TABLE IF NOT EXISTS products (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), price FLOAT, image_path VARCHAR(255))")

        # Créer la table sales si elle n'existe pas
        self.cursor.execute("CREATE TABLE IF NOT EXISTS sales (id INT AUTO_INCREMENT PRIMARY KEY, product_id INT, quantity INT, total_price FLOAT, sale_date DATE, vendor_id INT)")

        # Créer la table vendors si elle n'existe pas
        self.cursor.execute("CREATE TABLE IF NOT EXISTS vendors (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), email VARCHAR(255))")

        self.db.commit()

    def create_widgets(self):
        self.notebook = ttk.Notebook(self)
        self.notebook.pack(fill=tk.BOTH, expand=True)

        self.product_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.product_tab, text="Produits")
        self.create_product_widgets()

        self.sale_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.sale_tab, text="Ventes")
        self.create_sale_widgets()

        self.vendor_tab = ttk.Frame(self.notebook)
        self.notebook.add(self.vendor_tab, text="Vendeurs")
        self.create_vendor_widgets()

    def create_product_widgets(self):
        self.product_name_label = ttk.Label(self.product_tab, text="Nom du Produit:")
        self.product_name_label.pack()
        self.product_name_entry = ttk.Entry(self.product_tab)
        self.product_name_entry.pack()

        self.product_price_label = ttk.Label(self.product_tab, text="Prix du Produit:")
        self.product_price_label.pack()
        self.product_price_entry = ttk.Entry(self.product_tab)
        self.product_price_entry.pack()

        self.image_path_label = ttk.Label(self.product_tab, text="Chemin de l'Image:")
        self.image_path_label.pack()
        self.image_path_entry = ttk.Entry(self.product_tab)
        self.image_path_entry.pack()

        self.browse_button = ttk.Button(self.product_tab, text="Parcourir...", command=self.browse_image)
        self.browse_button.pack()

        self.add_product_button = ttk.Button(self.product_tab, text="Ajouter Produit", command=self.add_product)
        self.add_product_button.pack()

        self.product_tree = ttk.Treeview(self.product_tab, columns=("ID", "Nom", "Prix", "Image"))
        self.product_tree.heading("#0", text="ID")
        self.product_tree.heading("#1", text="Nom")
        self.product_tree.heading("#2", text="Prix")
        self.product_tree.heading("#3", text="Image")
        self.product_tree.pack()

        self.load_products()

    def browse_image(self):
        filepath = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")])
        self.image_path_entry.delete(0, tk.END)
        self.image_path_entry.insert(0, filepath)

    def load_products(self):
        self.product_tree.delete(*self.product_tree.get_children())
        self.cursor.execute("SELECT * FROM products")
        products = self.cursor.fetchall()
        for product in products:
            id, name, price, image_path = product
            self.product_tree.insert("", "end", values=(id, name, price, image_path))

    def add_product(self):
        name = self.product_name_entry.get()
        price = float(self.product_price_entry.get())
        image_path = self.image_path_entry.get()
        sql = "INSERT INTO products (name, price, image_path) VALUES (%s, %s, %s)"
        val = (name, price, image_path)
        self.cursor.execute(sql, val)
        self.db.commit()
        self.load_products()
        messagebox.showinfo("Success", "Product added successfully")

    def create_sale_widgets(self):
        self.product_id_label = ttk.Label(self.sale_tab, text="ID du Produit:")
        self.product_id_label.pack()
        self.product_id_entry = ttk.Entry(self.sale_tab)
        self.product_id_entry.pack()

        self.quantity_label = ttk.Label(self.sale_tab, text="Quantité:")
        self.quantity_label.pack()
        self.quantity_entry = ttk.Entry(self.sale_tab)
        self.quantity_entry.pack()

        self.product_price_label = ttk.Label(self.sale_tab, text="Prix du Produit:")
        self.product_price_label.pack()
        self.product_price_entry = ttk.Entry(self.sale_tab)
        self.product_price_entry.pack()

        self.sale_date_label = ttk.Label(self.sale_tab, text="Date de Vente:")
        self.sale_date_label.pack()
        self.sale_date_entry = ttk.Entry(self.sale_tab)
        self.sale_date_entry.pack()

        self.vendor_id_label = ttk.Label(self.sale_tab, text="ID du Vendeur:")
        self.vendor_id_label.pack()
        self.vendor_id_entry = ttk.Entry(self.sale_tab)
        self.vendor_id_entry.pack()

        self.record_sale_button = ttk.Button(self.sale_tab, text="Enregistrer Vente", command=self.record_sale)
        self.record_sale_button.pack()

        self.sales_tree = ttk.Treeview(self.sale_tab, columns=("ID", "Produit", "Quantité", "Prix Total", "Date de Vente", "ID Vendeur"))
        self.sales_tree.heading("#0", text="ID")
        self.sales_tree.heading("#1", text="Produit")
        self.sales_tree.heading("#2", text="Quantité")
        self.sales_tree.heading("#3", text="Prix Total")
        self.sales_tree.heading("#4", text="Date de Vente")
        self.sales_tree.heading("#5", text="ID Vendeur")
        self.sales_tree.pack()

        self.load_sales()

    def record_sale(self):
        product_id = int(self.product_id_entry.get())
        quantity = int(self.quantity_entry.get())
        total_price = quantity * float(self.product_price_entry.get())
        sale_date = self.sale_date_entry.get()
        vendor_id = int(self.vendor_id_entry.get())
        sql = "INSERT INTO sales (product_id, quantity, total_price, sale_date, vendor_id) VALUES (%s, %s, %s, %s, %s)"
        val = (product_id, quantity, total_price, sale_date, vendor_id)
        self.cursor.execute(sql, val)
        self.db.commit()
        messagebox.showinfo("Success", "Sale recorded successfully")
        self.load_sales()

    def load_sales(self):
        self.sales_tree.delete(*self.sales_tree.get_children())
        self.cursor.execute("SELECT * FROM sales")
        sales = self.cursor.fetchall()
        for sale in sales:
            id, product_id, quantity, total_price, sale_date, vendor_id = sale
            self.sales_tree.insert("", "end", values=(id, product_id, quantity, total_price, sale_date, vendor_id))

    def create_vendor_widgets(self):
        self.vendor_name_label = ttk.Label(self.vendor_tab, text="Nom du Vendeur:")
        self.vendor_name_label.pack()
        self.vendor_name_entry = ttk.Entry(self.vendor_tab)
        self.vendor_name_entry.pack()

        self.vendor_email_label = ttk.Label(self.vendor_tab, text="Email du Vendeur:")
        self.vendor_email_label.pack()
        self.vendor_email_entry = ttk.Entry(self.vendor_tab)
        self.vendor_email_entry.pack()

        self.add_vendor_button = ttk.Button(self.vendor_tab, text="Ajouter Vendeur", command=self.add_vendor)
        self.add_vendor_button.pack()

        self.vendor_tree = ttk.Treeview(self.vendor_tab, columns=("ID", "Nom", "Email"))
        self.vendor_tree.heading("#0", text="ID")
        self.vendor_tree.heading("#1", text="Nom")
        self.vendor_tree.heading("#2", text="Email")
        self.vendor_tree.pack()

        self.load_vendors()

    def add_vendor(self):
        name = self.vendor_name_entry.get()
        email = self.vendor_email_entry.get()
        sql = "INSERT INTO vendors (name, email) VALUES (%s, %s)"
        val = (name, email)
        self.cursor.execute(sql, val)
        self.db.commit()
        self.load_vendors()
        messagebox.showinfo("Success", "Vendor added successfully")

    def load_vendors(self):
        self.vendor_tree.delete(*self.vendor_tree.get_children())
        self.cursor.execute("SELECT * FROM vendors")
        vendors = self.cursor.fetchall()
        for vendor in vendors:
            id, name, email = vendor
            self.vendor_tree.insert("", "end", values=(id, name, email))

if __name__ == "__main__":
    app = App()
    app.mainloop()


ValueError: not enough values to unpack (expected 6, got 4)